## DFF and TFF (Toggle Flip-Flop)

In this example we create a toggle flip-flop (TFF) from a d-flip-flop (DFF). In `Magma`, finite state machines can be constructed by composing combinational logic with flop-flops register primitives.

In [1]:
import magma as m
from mantle import DFF

/Users/hanrahan/miniconda3/lib/python3.7/site-packages/pysmt/walkers/generic.py:43: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if len(nodetypes) == 1 and isinstance(nodetypes[0], collections.Iterable):


In [2]:
class TFF(m.Circuit):
    IO = ['O', m.Out(m.Bit)] + m.ClockInterface()
    @classmethod
    def definition(io):
        # instance a dff to hold the state of the toggle flip-flop - this needs to be done first
        dff = DFF()
        # compute the next state as the not of the old state ff.O
        io.O <= dff(~dff.O)
        #m.wiredefaultclock(io, dff)
        #m.wireclock(io, dff)
        
def tff():
    return TFF()()

Test using the python simulator.

In [3]:
from magma.simulator import PythonSimulator

sim = PythonSimulator(TFF, TFF.CLK)

sim.evaluate()
val = sim.get_value(TFF.O)
assert val == False

for i in range(10):
    val = not val
    
    sim.advance() # toggle clock - now High
    assert val == sim.get_value(TFF.O)

    sim.advance() # toggle clock - now Low
    assert val == sim.get_value(TFF.O)

print("Success!")

ValueError: Cannot simulate magma_Bit_not_inst0<magma_Bit_not(I: In(Bit), O: Out(Bit))> of type magma_Bit_not(I: In(Bit), O: Out(Bit)) because it does not have a Python simulate method

### Generate Verilog

In [ ]:
m.compile("build/TFF", TFF, output="verilog")
%cat build/TFF.v

Generate verilog with `coreir` modules included.

In [ ]:
m.compile("build/TFF", TFF, output="coreir-verilog")
%cat build/TFF.v

In [ ]:
m.compile("build/TFF", TFF, output="coreir")
%cat build/TFF.json

In [ ]:
!coreir -i build/TFF.json -p instancecount